<a href="https://colab.research.google.com/github/nschmidtg/thesis/blob/main/Test1_as_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In Google Colab: Be sure to select a GPU runtime (Runtime → Change runtime type → Hardware accelarator).


In [ ]:
# First off, install asteroid
!pip install git+https://github.com/asteroid-team/asteroid --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


## After installing requirements, you need to Restart Runtime (Ctrl + M).
Else it will fail to import asteroid

In [ ]:
!pip install pytorch-lightning --quiet

     |████████████████████████████████| 276kB 13.0MB/s 


In [ ]:
# Asteroid is based on PyTorch and PyTorch-Lightning.
from torch import optim
from pytorch_lightning import Trainer

In [ ]:
# We train the same model architecture that we used for inference above.
from asteroid import DPRNNTasNet

In [ ]:
# In this example we use Permutation Invariant Training (PIT) and the SI-SDR loss.
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper

In [ ]:
# install musdb:
!pip install musdb --quiet

     |████████████████████████████████| 512kB 13.0MB/s 


In [ ]:
# install ffmpeg (stems are mp4 by default)
!sudo apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
# MiniLibriMix is a tiny version of LibriMix (https://github.com/JorisCos/LibriMix),
# which is a free speech separation dataset.
from asteroid.data import LibriMix

# import musdb to create the mixtures: https://github.com/sigsep/sigsep-mus-db
import musdb
# Asteroid's System is a convenience wrapper for PyTorch-Lightning.
from asteroid.engine import System

In [ ]:
# download the musdb library
mus = musdb.DB(download=True)

# To use the full dataset, set a dataset root directory
# mus = musdb.DB(root="/path/to/musdb)

# To work directly with wav: https://github.com/sigsep/sigsep-mus-db#using-wav-files-optional

Done!


array([[-8.84704590e-02, -3.79333496e-02],
       [-1.07421875e-01, -5.22460938e-02],
       [-8.42590332e-02, -5.48400879e-02],
       ...,
       [-8.23974609e-04,  3.05175781e-05],
       [-8.54492188e-04,  7.01904297e-04],
       [-8.54492188e-04,  1.31225586e-03]])

In [ ]:
# get the first track
track = mus[0]

# get just the vocals
stereo_track = track.targets['vocals'].audio

# get the mix
stereo_track = track.audio

# split sources
left_track = stereo_track[:,0]
right_track = stereo_track[:,1]

# mono mix
mono_track = left_track + right_track

In [ ]:
# play mono track
display(Audio(mono_track, rate = 44100))

In [ ]:
# play stereo track
display(Audio([left_track, right_track], rate = 44100))

In [ ]:
# This will automatically download MiniLibriMix from Zenodo on the first run.
train_loader, val_loader = LibriMix.loaders_from_mini(task="sep_clean", batch_size=8)


Drop 0 utterances from 800 (shorter than 3 seconds)
Drop 0 utterances from 200 (shorter than 3 seconds)


# Create the augmented dataset

using the LibriMix and the Musdb18 datasets, an augmented podcast/radioshow like dataset is created

In [ ]:
import librosa, os

In [ ]:
def create_folder_structure(path):
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(path + '/linear_mono'):
        os.makedirs(path + '/linear_mono')
    if not os.path.exists(path + '/linear_stereo'):
        os.makedirs(path + '/linear_stereo')
    if not os.path.exists(path + '/sidechain_mono'):
        os.makedirs(path + '/sidechain_mono')
    if not os.path.exists(path + '/sidechain_stereo'):
        os.makedirs(path + '/sidechain_stereo')
    if not os.path.exists(path + '/track'):
        os.makedirs(path + '/track')
    if not os.path.exists(path + '/speech'):
        os.makedirs(path + '/speech')

In [ ]:
# create files structure
train_path = "augmented_dataset/train"
create_folder_structure(train_path)

test_path = "augmented_dataset/test"
create_folder_structure(test_path)

In [ ]:
def mix_audio_sources(track, speech, output_path, music_to_speech_ratio = 0.8):
    """
    Creates 4 mixes for the a music and a speech track and locates it in the output_path
    the 4 mixes are: linear_mono, linear_stereo, sidechain_mono, sidechain_stereo
    """
    # read the audio
    track, fs_track = librosa.load(track_path)
    speech, fs_speech = librosa.load(speech_path)

    # no need to resample since librosa does that for us      
    # match the length of the files
    min_lenght = min(len(track), len(speech))
    
    # crop the files to match in length
    song = song[0:min_lenght]
    speech = speech[0:min_lenght]
    
    linear_mono = 

# Train the network

In [ ]:
# Tell DPRNN that we want to separate to 2 sources.
model = DPRNNTasNet(n_src=2)

In [ ]:
# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

In [ ]:
# Train for 1 epoch using a single GPU. If you're running this on Google Colab,
# be sure to select a GPU runtime (Runtime → Change runtime type → Hardware accelarator).
trainer = Trainer(max_epochs=1, gpus=1)
trainer.fit(system)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type           | Params
---------------------------------------------
0 | model     | DPRNNTasNet    | 3 M   
1 | loss_func | PITLossWrapper | 0     


Saving latest checkpoint..


1

In [ ]:
!pip install librosa --quiet

In [ ]:
%%capture
!wget https://www.merl.com/demos/deep-clustering/media/female-female-mixture.wav

In [ ]:
import librosa

# You can pass a NumPy array:
mixture, _ = librosa.load("female-female-mixture.wav", sr=8000)
model.separate(mixture)

# Or simply a file name:
model.separate("female-female-mixture.wav")

/usr/local/lib/python3.6/dist-packages/asteroid/separate.py:162: UserWarning: File female-female-mixture_est1.wav already exists, pass `force_overwrite=True` to overwrite it
  UserWarning,


In [ ]:
from IPython.display import display, Audio

display(Audio("female-female-mixture_est1.wav"))

display(Audio("female-female-mixture_est2.wav"))

ValueError: ignored

In [ ]:
s1, _ = librosa.load("female-female-mixture_es1.wav", sr=8000)


# Try to use ConvTasNet


In [ ]:
from asteroid import ConvTasNet

In [ ]:
# Tell DPRNN that we want to separate to 2 sources.
model = ConvTasNet(n_src=2)

In [ ]:
# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

In [ ]:
# Train for 1 epoch using a single GPU. If you're running this on Google Colab,
# be sure to select a GPU runtime (Runtime → Change runtime type → Hardware accelarator).
trainer = Trainer(max_epochs=1, gpus=1)
trainer.fit(system)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type           | Params
---------------------------------------------
0 | model     | ConvTasNet     | 5 M   
1 | loss_func | PITLossWrapper | 0     


Saving latest checkpoint..


1